#Notebook to process shape files and get compactness score for each district
Author: Tatsiana Maskalevich

In [ ]:
import matplotlib.pyplot as plt
import pysal as ps
import numpy as np
import pandas as pd
import math
import json

In [ ]:
def compute_compactness(input_file_path, output_json_filepath, output_csv_file_path):
    '''
        Method to compute compactness and write output to disk in json and csv format
        input:
        input_file_path - path to shp file
        output_json_filepath - path to store json file with GEOID to compactness mapping
        output_csv_file_path - path to store csv file with info from Tiger files (dbf) plus compactness
        
    '''
    # open shape file
    shp = ps.open(input_file_path)
    area = []
    perimeter = []
    # get polygon from a shape file and 
    for poly in shp:
        area.append(poly.area)
        perimeter.append(poly.perimeter)

    #read dbf file 
    dbf = ps.open(input_file_path.replace('.shp', '.dbf'))
    d = dict([(col, np.array(dbf.by_col(col))) for col in dbf.header])
    df = pd.DataFrame(d)
    # add area, perimeter, and compactness to the main DF
    df['AREA'] = area
    df['PERIMETER'] = perimeter
    df['COMPACTNESS'] = [np.round(a*math.pi*4/b**2,5) for a,b in zip(area,perimeter)]  
    #write compactness by geoid into json  and csv formats 

    #to json
    compact_dict = dict(zip(df['GEOID'], df['COMPACTNESS']))
    
    with open(output_json_filepath, 'wb') as f:
        print "Writing " + output_json_filepath + " to disk"
        f.write(json.dumps(compact_dict))
    #to csv
    print "Writing " + output_csv_file_path + " to disk"
    df.to_csv(output_csv_file_path, sep=',')


In [ ]:
# define location of the shape file
file2013 = '../data/shapefiles/tl_rd13_us_cd113/tl_rd13_us_cd113.shp'

output_filepath = '../data/'

compute_compactness(file2013, output_filepath+"compactness113_byGEO.json", output_filepath+"cd113_data_complete.csv")